In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch torchvision
!pip install yolov5

In [ ]:
import torch
from PIL import Image
import cv2
import json
import os
from sklearn.metrics import accuracy_score
from pycocotools.coco import COCO
# 加载预训练的 YOLOv5x 模型
model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)

coco = COCO('/content/drive/MyDrive/train200.json')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirement ['pillow>=10.3.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 16.0 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0

requirements: AutoUpdate success ✅ 10.7s, installed 1 package: ['pillow>=10.3.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-5-16 Python-3.10.12 torch-2.2.1+cu121 CPU

100%|██████████| 166M/166M [00:02<00:00, 76.5MB/s]

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients, 205.5 GFLOPs
Adding AutoShape... 


loading annotations into memory...
Done (t=0.88s)
creating index...
index created!


In [ ]:
# 获取所有类别
categories = coco.loadCats(coco.getCatIds())

# 打印类别名称
for category in categories:
    print(category['id'], category['name'])


1 boat


In [ ]:
import os
import cv2
from pycocotools.coco import COCO

# 初始化 COCO api
coco = COCO('/content/drive/MyDrive/train200.json')  # JSON 注释文件路径

# 感兴趣的类别
interested_categories = ['boat']

# 获取这些类别的ID
catIds = coco.getCatIds(catNms=interested_categories)

# 分别获取每个类别的图像ID
imgIds = coco.getImgIds(catIds=catIds)

# 创建一个目录来保存下载的图像
os.makedirs('/content/drive/MyDrive/200images', exist_ok=True)

# 按照图像ID顺序下载图像
for imgId in imgIds:
    img_info = coco.loadImgs(imgId)[0]

    # 构建图像的本地文件路径
    for subfolder in ['Day', 'Night']:
        local_img_path = os.path.join('/content/drive/MyDrive/200', subfolder, img_info['file_name'])

        # 如果在当前子文件夹找到了图像，则进行处理
        if os.path.isfile(local_img_path):
            img = cv2.imread(local_img_path)
            if img is not None:
                # 写入图像到指定路径
                cv2.imwrite(os.path.join('/content/drive/MyDrive/200images', img_info['file_name']), img)
            else:
                print(f"无法加载图像 {img_info['file_name']}")
            break  # 图像找到后，不需要继续检查其他子文件夹

print("图像处理完成。")


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
图像处理完成。


In [ ]:
import torch
from pycocotools.coco import COCO
import os
import cv2
import json

thing_classes = ['boat']

def predict_with_yolov5(img_dir, json_file, categories):
    model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)
    coco_api = COCO(json_file)
    cat_ids = coco_api.getCatIds(catNms=categories)
    img_ids = coco_api.getImgIds(catIds=cat_ids)
    predictions = []

    for img_id in img_ids:
        img_info = coco_api.loadImgs(img_id)[0]
        img_path = os.path.join(img_dir, img_info['file_name'])

        try:
            image = cv2.imread(img_path)
            if image is not None:
                # OpenCV 读取的图像通道顺序为 BGR，需要转换为 RGB
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = model(image_rgb)

                # 将 DataFrame 转换为字典
                for result in results.pandas().xyxy[0].to_dict(orient='records'):
                    predictions.append({
                        "image_id": img_id,
                        "category_id": cat_ids[0],  # Assuming only one category (Boat)
                        "bbox": [result['xmin'], result['ymin'], result['xmax'] - result['xmin'], result['ymax'] - result['ymin']],
                        "score": result['confidence']
                    })
            else:
                print(f"无法加载图像 {img_path}")
        except Exception as e:
            print(f"Error processing {img_path}: {e}")

    return predictions

img_dir = "/content/drive/MyDrive/200images"
json_file = "/content/drive/MyDrive/train200.json"
predictions = predict_with_yolov5(img_dir, json_file, thing_classes)

# 保存预测结果为JSON文件
with open('/content/drive/MyDrive/200.json', 'w') as f:
    json.dump(predictions, f)

# 加载预测结果和真值进行评估
from pycocotools.cocoeval import COCOeval

# 加载预测结果和真值
coco_gt = COCO(json_file)
coco_dt = coco_gt.loadRes('/content/drive/MyDrive/200.json')

# 初始化COCOeval对象
coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')

# 选择要评估的类别
cat_ids = coco_gt.getCatIds(catNms=thing_classes)
coco_eval.params.catIds = cat_ids

# 运行COCOeval
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-5-16 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients, 205.5 GFLOPs
Adding AutoShape... 


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.10s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.431
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.593
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.513
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.032
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.452
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.341
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | max